In [2]:
cd ../../task_01/pipelines

/Users/mathieuchevalley/Documents/Study/ETHZ/Fall Semester 2020/Data Science lab/data_sc_lab/task_01/pipelines


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipynb.fs.full.tool_functions import *
from yellowbrick.regressor import AlphaSelection

In [4]:
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')#, nrows=1000)

df_spectro_cluster = pd.read_csv("../../../data/EEG/RestingEEG_Spectro_Cluster.csv") #, nrows=1000)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103

In [6]:
y_labels = df_beha[["Patient_ID", "Age", "DX_01"]]
healt_ind = (y_labels["DX_01"].str.contains("No Diagnosis Given") == True)
ADHD_ind = (y_labels["DX_01"].str.contains('ADHD') == True)
keep = healt_ind | ADHD_ind

In [10]:
df_beha = df_beha[keep]
df_spectro_cluster = df_spectro_cluster[keep]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [11]:
s_in = df_beha[['SWAN_IN_Avg' , 'Age', 'Patient_ID']]
s_HY = df_beha[['SWAN_HY_Avg' , 'Age', 'Patient_ID']]

In [12]:
s_in

,SWAN_IN_Avg,Age,Patient_ID
2,1.444444,10.052589,NDARRM363BXZ
8,1.111111,17.203855,NDARED632KNG
9,2.222222,14.216632,NDARCF462HNM
11,0.777777,9.153661,NDARFC233RND
22,1.222222,17.510381,NDARDT800YVF
...,...,...,...
2089,0.000000,17.419917,NDARZC497BFU
2090,NaN,13.031143,NDARDZ737LG9
2091,-0.666666,7.563084,NDARYZ986HEW
2094,NaN,10.712069,NDARRT283RHB


In [13]:
s_in['SWAN_IN_Avg'].isnull().sum()

265

In [14]:
s_HY['SWAN_HY_Avg'].isnull().sum()

265

In [15]:
s = df_beha[['SWAN_IN_Avg' , 'SWAN_HY_Avg', 'Age', 'Patient_ID']]

In [16]:
s = s[~s_in['SWAN_IN_Avg'].isnull()]

In [17]:
df_spectro_cluster.rename(columns={'id': 'Patient_ID'}, inplace=True)
df_label_spectro_cluster = pd.merge(s, df_spectro_cluster, on='Patient_ID')

df_label_spectro_cluster = fill_with_median(df_label_spectro_cluster)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [18]:
less_18 = df_label_spectro_cluster['Age'] <= 18.0
df_label_spectro_cluster = df_label_spectro_cluster[less_18]

In [19]:
df_label_spectro_cluster

,SWAN_IN_Avg,SWAN_HY_Avg,Age,Patient_ID,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,eyesclosed_fband_delta_absmean_rfront,eyesclosed_fband_delta_absmean_lpari,eyesclosed_fband_delta_absmean_mpari,eyesclosed_fband_delta_absmean_rpari,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,1.111111,0.555555,10.758156,NDARYP516VUU,3.736577,5.970049,3.462826,4.225793,6.973617,6.719936,...,10.750797,10.401234,10.783732,11.046665,0.821805,0.784304,0.770845,0.914380,0.671399,1.004923
1,0.888888,-2.111111,12.357403,NDARLE953JYU,3.519179,5.179898,3.495491,3.629303,4.704654,2.793679,...,11.078667,10.814422,10.176920,11.290443,0.718186,0.358765,0.504265,0.691663,0.568579,0.563663
2,-0.555555,0.222222,16.039356,NDARVN469KP6,1.551702,2.188666,1.582556,2.293936,4.226975,1.248253,...,8.568319,8.717960,8.949643,8.751536,0.419412,0.384375,0.345518,0.335640,0.226452,0.496402
3,1.333333,1.666666,8.784280,NDARZY019GNZ,6.233496,6.755708,9.088227,9.852386,14.023665,14.113028,...,7.349352,7.500019,7.532849,7.426925,0.829479,0.856743,0.881540,0.848050,1.112069,0.823531
4,1.111111,0.777777,7.218571,NDARHZ439EBY,5.593511,7.239707,5.585892,14.176308,17.896530,5.948757,...,8.689175,8.533435,8.744619,8.605137,0.862066,0.304196,0.846766,1.123680,1.078171,1.022903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,2.777777,3.000000,10.454825,NDARAV187GJ5,13.432350,30.785647,11.838032,10.256610,10.570761,8.799888,...,6.129422,9.855379,5.871590,5.866629,0.120608,0.141840,0.088540,0.436530,0.126118,0.126822
365,1.222222,-1.000000,14.610312,NDARGH592NZ2,2.854754,3.859619,4.131474,5.286843,6.619541,6.451419,...,9.938448,10.089338,8.315127,7.965627,0.386374,0.760298,0.842317,0.975704,0.489352,0.326751
366,-3.000000,-3.000000,10.046315,NDARRV505ND6,8.807898,8.647452,5.419540,7.672813,16.092883,7.072346,...,11.039922,11.104643,10.824285,11.067008,0.220845,0.176626,0.354522,0.459443,0.275766,0.379102
367,0.222222,0.111111,13.022359,NDARKN175HWB,5.877912,5.186567,5.388318,7.797174,5.577393,4.532206,...,16.499717,14.178789,12.342108,12.185766,0.113648,0.556684,0.570148,0.559704,0.436026,0.361575


In [18]:
less_18 = df_label_spectro_cluster['Age'] <= 18.0
df_label_spectro_cluster = df_label_spectro_cluster[less_18]


In [20]:
outliers = pd.read_csv('../../../data/outliers.csv')
df_label_spectro_cluster = df_label_spectro_cluster[~df_label_spectro_cluster.Patient_ID.isin(outliers.Patient_ID)]

In [41]:
from sklearn.model_selection import train_test_split
x_train , x_test, y_train , y_test = \
train_test_split(df_label_spectro_cluster.drop([ 'SWAN_IN_Avg' , 'SWAN_HY_Avg', 'Patient_ID'], axis=1), df_label_spectro_cluster[['SWAN_IN_Avg', 'SWAN_HY_Avg']], test_size=0.2, random_state = 0)


In [42]:
forest = IsolationForest()
forest.fit(x_train)

# Outlier indices for training
outliers_training = forest.predict(x_train)
outliers_training_indices = np.argwhere(outliers_training == 1).flatten()

# Drop signal outliers in training data
x_train = x_train[outliers_training == 1]
y_train = y_train[outliers_training == 1]
#x_train, y_train = drop_outliers_samples_isolation_forest(x_train, y_train)
print(outliers_training_indices)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


[  0   1   2   3   4   5   6   7   8  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  48  49  50  51  52  54  55  56  57
  58  59  60  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  78  79  80  81  82  83  84  85  86  87  88  89  90  92  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 112 113 114 115 116
 117 118 119 120 121 122 124 125 126 128 129 130 133 134 137 138 139 140
 141 143 144 145 146 147 148 150 151 152 153 154 155 158 159 160 161 162
 163 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 190 191 192 193 194 197 198 199 200 201 202
 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220
 221 222 223 224 226 227 228 229 230 231 232 233 234 236 237 238 239 240
 241 242 243 245 246 247 249 250 251 252 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 269 270 271 272 273 274 27

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [43]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

(73, 301)
(73, 2)
(262, 301)
(262, 2)


In [22]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

In [28]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(5, input_dim=n_inputs, activation='relu'))
    model.add(Dense(5, activation='relu'))
    #model.add(Dense(5, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], 1#y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        file_path = "model.h5"
        model = get_model(n_inputs, n_outputs)
        # fit model
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        model.fit(X_train, y_train, verbose=0, callbacks=callbacks_list, validation_split=0.1,  epochs=150)
        # evaluate model on test set
        mse = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mse)
        results.append(mse)
    return results

In [29]:
results = evaluate_model(x_train.to_numpy(), y_train.to_numpy()[:,0])
print('MSE: %.3f (%.3f)' % (mean(results), std(results)))

>1.295
>1.465
>0.692
>1.557
>1.923
>1.443
>0.744
>1.535
>1.646
>2.042
MSE: 1.434 (0.416)


In [30]:
results = evaluate_model(x_train.to_numpy(), y_train.to_numpy()[:,1])
print('MSE: %.3f (%.3f)' % (mean(results), std(results)))

>1.099
>1.674
>1.486
>1.455
>1.631
>1.582
>1.446
>1.401
>1.755
>1.761
MSE: 1.529 (0.188)


In [31]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
def evaluate_model_10(X, y, x_test, y_test):
    results = list()
    for i in range(10):
        n_inputs, n_outputs = X.shape[1], 1
        model = get_model(n_inputs, n_outputs)
        file_path = "model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
        #early = EarlyStopping(monitor="val_loss", mode="min", patience=10, verbose=1)
        #redonplat = ReduceLROnPlateau(monitor="val_loss", mode="min", patience=10, verbose=2)
        callbacks_list = [checkpoint] 
        model.fit(X, y, callbacks=callbacks_list, verbose=0,validation_split=0.1,  epochs=150)
        model.load_weights(file_path)
        mse = model.evaluate(x_test, y_test, verbose=2)
        print('>%.3f' % mse)
        results.append(mse)
    print('MSE: %.3f (%.3f)' % (mean(results), std(results)))

In [32]:
evaluate_model_10(x_train.to_numpy(), y_train.to_numpy()[:, 0], x_test.to_numpy(), y_test.to_numpy()[:,0])

3/3 - 0s - loss: 1.3399
>1.340
3/3 - 0s - loss: 1.3481
>1.348
3/3 - 0s - loss: 2.2819
>2.282
3/3 - 0s - loss: 1.2372
>1.237
3/3 - 0s - loss: 1.4086
>1.409
3/3 - 0s - loss: 1.2945
>1.294
3/3 - 0s - loss: 1.4301
>1.430
3/3 - 0s - loss: 1.4364
>1.436
3/3 - 0s - loss: 1.3474
>1.347
3/3 - 0s - loss: 1.2393
>1.239
MSE: 1.436 (0.290)


In [33]:
evaluate_model_10(x_train.to_numpy(), y_train.to_numpy()[:, 1], x_test.to_numpy(), y_test.to_numpy()[:,1])

3/3 - 0s - loss: 1.7450
>1.745
3/3 - 0s - loss: 1.6595
>1.659
3/3 - 0s - loss: 1.5512
>1.551
3/3 - 0s - loss: 1.7846
>1.785
3/3 - 0s - loss: 1.7867
>1.787
3/3 - 0s - loss: 1.5846
>1.585
3/3 - 0s - loss: 1.5916
>1.592
3/3 - 0s - loss: 1.5783
>1.578
3/3 - 0s - loss: 1.5015
>1.501
3/3 - 0s - loss: 1.5272
>1.527
MSE: 1.631 (0.101)


In [178]:
sklearn.metrics.mean_squared_error(model.predict(x_test.to_numpy())[:,0], y_test.to_numpy()[:,0])

1.5197307342097406

In [36]:
import sklearn
sklearn.metrics.mean_squared_error(model.predict(x_test.to_numpy()), y_test.to_numpy()[:,1])

NameError: name 'model' is not defined

In [39]:
import sklearn
y = np.zeros(len(y_test))
y = y + np.mean(y_train.to_numpy()[:,0])
sklearn.metrics.mean_squared_error(y, y_test.to_numpy()[:,0])

1.2407459939026044

In [40]:
y = np.zeros(len(y_test))
y = y + np.mean(y_train.to_numpy()[:,1])
sklearn.metrics.mean_squared_error(y, y_test.to_numpy()[:,1])

1.6361967270979005

In [69]:
model.predict(x_test.to_numpy())

array([[-8.88524294e-01, -1.61943531e+00],
       [ 1.26003444e-01, -5.88641644e-01],
       [ 5.66600323e-01,  6.48859143e-02],
       [ 1.03167117e+00,  5.36886394e-01],
       [ 1.03228629e+00,  5.35459280e-01],
       [ 1.03228629e+00,  5.35459280e-01],
       [ 1.02762544e+00,  5.46271801e-01],
       [ 2.62520969e-01, -2.56546259e-01],
       [-9.21010375e-01, -9.01059389e-01],
       [ 8.72042000e-01,  4.49065983e-01],
       [ 9.86840427e-01,  6.40888214e-01],
       [ 9.55724716e-01,  7.13072836e-01],
       [ 2.85151303e-01,  9.58248615e-01],
       [ 1.03228629e+00,  5.35459280e-01],
       [ 3.65076840e-01,  2.08330441e+00],
       [ 8.01041782e-01,  1.07191849e+00],
       [-3.38420033e-01,  3.71533227e+00],
       [ 7.83345819e-01,  1.11297107e+00],
       [ 1.02798593e+00,  5.28667331e-01],
       [-1.90740824e-02, -7.17217088e-01],
       [ 5.25200546e-01,  1.47220278e+00],
       [ 1.03228629e+00,  5.35459280e-01],
       [ 1.02435327e+00,  5.53862989e-01],
       [ 8.

In [44]:
def fit_xgboost_with_grid_search(X_train, y_train):

    # Create gradient boosting object
    xgbr = XGBRegressor(verbosity=1, max_depth=10, reg_lambda=1) 
    
    # Grid search 
    parameters = {'max_depth':[1,2,4], 'reg_lambda':[4, 8, 16,  20], 'min_child_weight':[6,  12, 25, 30]}
    clf = GridSearchCV(xgbr, parameters, scoring='neg_mean_squared_error', n_jobs=4, iid=False, cv=5)
    clf.fit(X_train, y_train)
    
    print("Best parameters for XGBoost: " + str(clf.best_params_))
            
    return clf

In [45]:
rdf = fit_xgboost_with_grid_search(x_train, y_train.to_numpy()[:,0])
rdf.best_score_

Best parameters for XGBoost: {'max_depth': 2, 'min_child_weight': 6, 'reg_lambda': 4}


-1.4355690593672894

In [46]:
sklearn.metrics.mean_squared_error(rdf.predict(x_test), y_test.to_numpy()[:,0])

2.1016905252985283

In [47]:
rdf = fit_xgboost_with_grid_search(x_train, y_train.to_numpy()[:,1])
rdf.best_score_

Best parameters for XGBoost: {'max_depth': 1, 'min_child_weight': 30, 'reg_lambda': 20}


-1.5470889671844368

In [48]:
sklearn.metrics.mean_squared_error(rdf.predict(x_test), y_test.to_numpy()[:,1])

1.8034752258857152

In [49]:
from sklearn import svm
svr= svm.SVR()
parameters = {'kernel' : ( 'rbf' ,  'sigmoid' ), 'C':[0.1, 1,10,100], 'gamma':['auto']}
clf = GridSearchCV(svr, parameters ,cv =5, scoring='neg_mean_squared_error')
clf.fit(x_train , y_train.to_numpy()[:,0])
clf.best_score_

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-1.3367770569691524

In [50]:
sklearn.metrics.mean_squared_error(clf.predict(x_test.to_numpy()), y_test.to_numpy()[:,0])

1.2618694690052206

In [51]:
from sklearn import svm
svr= svm.SVR()
parameters = {'kernel' : ( 'rbf' ,  'sigmoid' ), 'C':[0.1, 1,10,100], 'gamma':['auto']}
clf = GridSearchCV(svr, parameters ,cv =5, scoring='neg_mean_squared_error')
clf.fit(x_train , y_train.to_numpy()[:,1])
clf.best_score_

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-1.3992035558888072

In [52]:
sklearn.metrics.mean_squared_error(clf.predict(x_test.to_numpy()), y_test.to_numpy()[:,1])

1.5273206456753292